In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

c:\users\heythere\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
mnist = input_data.read_data_sets("../../Tensorflow-Bootcamp-master/03-Convolutional-Neural-Networks/MNIST_data/",one_hot=True)

In [ ]:
tf.reset_default_graph() 

In [ ]:
# DEFINE parameters
num_label = 10
label_hid1 = 5
label_hid2 = 3
label_hid3 = 2
label_hid4 = 1

num_image = 784 # 28*28
image_hid1 = 392
image_hid2 = 196
image_hid3 = 98
image_hid4 = 49
image_hid5 = 25
image_hid6 = 12
image_hid7 = 6
image_hid8 = 3
image_hid9 = 2
image_hid10 = 1

In [ ]:
tie_weight = tf.transpose
act_func = lambda X:X
initializer = tf.variance_scaling_initializer()

In [ ]:
## PLACEHOLDER DEFINATION
X = tf.placeholder(tf.float32, shape=[None, num_image])
Y = tf.placeholder(tf.float32, shape=[None, num_label])

## Loading Encoded Model

In [ ]:
## IMAGE COMPRESSION
wi1 = tf.Variable(np.load('../save/weights/w1.npy'))
wi2 = tf.Variable(np.load('../save/weights/w2.npy'))
wi3 = tf.Variable(np.load('../save/weights/w3.npy'))
wi4 = tf.Variable(np.load('../save/weights/w4.npy'))
wi5 = tf.Variable(np.load('../save/weights/w5.npy'))
wi6 = tf.Variable(np.load('../save/weights/w6.npy'))
wi7 = tf.Variable(np.load('../save/weights/w7.npy'))
wi8 = tf.Variable(np.load('../save/weights/w8.npy'))
wi9 = tf.Variable(np.load('../save/weights/w9.npy'))
wi10 = tf.Variable(np.load('../save/weights/w10.npy'))

In [ ]:
img_layer1 = act_func(tf.matmul(X, wi1))
img_layer2 = act_func(tf.matmul(img_layer1, wi2))
img_layer3 = act_func(tf.matmul(img_layer2, wi3))
img_layer4 = act_func(tf.matmul(img_layer3, wi4))
img_layer5 = act_func(tf.matmul(img_layer4, wi5))
img_layer6 = act_func(tf.matmul(img_layer5, wi6))
img_layer7 = act_func(tf.matmul(img_layer6, wi7))
img_layer8 = act_func(tf.matmul(img_layer7, wi8))
img_layer9 = act_func(tf.matmul(img_layer8, wi9))
img_layer10 = act_func(tf.matmul(img_layer9, wi10))

In [ ]:
## LABEL DECOMPRESSION
wl1 = tf.Variable(np.load('../save/weights/lw1.npy'))
wl2 = tf.Variable(np.load('../save/weights/lw2.npy'))
wl3 = tf.Variable(np.load('../save/weights/lw3.npy'))
wl4 = tf.Variable(np.load('../save/weights/lw4.npy'))

In [ ]:
lbl_layer1 = act_func(tf.matmul(Y, wl1))
lbl_layer2 = act_func(tf.matmul(lbl_layer1, wl2))
lbl_layer3 = act_func(tf.matmul(lbl_layer2, wl3))
lbl_layer4 = act_func(tf.matmul(lbl_layer3, wl4))

## Setting up learning Model

In [185]:
# Learning Parameters AKA neurogenesis
# w11 = tf.Variable(initializer([image_hid10, label_hid4]), dtype=tf.float32)

w62 = tf.constant(np.load('../save/weights/ngw6-2.npy'))

w63 = tf.Variable(initializer([image_hid7, label_hid2]), dtype=tf.float32)
w63 = tf.Variable(tf.matmul(w62, tf.transpose(wl3)))

In [187]:
inpt = img_layer7
output = tf.matmul(inpt,w63)
yval = lbl_layer2

In [188]:
tf.transpose(wl3)

<tf.Tensor 'transpose_7:0' shape=(2, 3) dtype=float32>

## Create Encode-Decode Model

In [189]:
# out_layer1 = tf.matmul(output,tf.transpose(wl4))
# out_layer2 = tf.matmul(output,tf.transpose(wl3))
out_layer3 = tf.matmul(output,tf.transpose(wl2))
out_layer4 = tf.matmul(out_layer3,tf.transpose(wl1))

In [190]:
# LOSS FUNCTIONS
loss = tf.reduce_mean(tf.square(output - yval))
# loss = tf.reduce_mean(tf.abs(output_layer - X))
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=yout))

In [191]:
# TRAINING
learning_rate = 0.0001
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss, var_list=[w63])

In [192]:
# PREPARATION
init = tf.global_variables_initializer()
saver = tf.train.Saver() 

In [199]:
num_epochs = 100
batch_size = 950

try:
    with tf.Session() as sess:
        sess.run(init)

        # Epoch == Entire Training Set
        for epoch in range(num_epochs):
            num_batches = mnist.train.num_examples // batch_size

            # 150 batch size
            for iteration in range(num_batches):
                X_batch, y_batch = mnist.train.next_batch(batch_size)
                sess.run(train, feed_dict={X: X_batch, Y: y_batch})

            training_loss = loss.eval(feed_dict={X: X_batch, Y: y_batch})   
            print("Epoch {} Complete. Training Loss: {}".format(epoch,training_loss))
            saver.save(sess, "../save/neuro_6-3.ckpt")
            
            matches = tf.equal(tf.argmax(out_layer4, 1), tf.argmax(Y,1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))

            print('TEST ACCURACY: ')
            print (sess.run(acc, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))
            print()
    
except KeyboardInterrupt:
    print('Keyboard Interrupted')
finally:
    print('Finished Training the Model')

Epoch 0 Complete. Training Loss: 9.785619735717773
TEST ACCURACY: 
0.2076

Epoch 1 Complete. Training Loss: 9.626299858093262
TEST ACCURACY: 
0.216

Epoch 2 Complete. Training Loss: 9.500679969787598
TEST ACCURACY: 
0.221

Epoch 3 Complete. Training Loss: 9.575742721557617
TEST ACCURACY: 
0.228

Epoch 4 Complete. Training Loss: 9.3663911819458
TEST ACCURACY: 
0.2338

Epoch 5 Complete. Training Loss: 9.618182182312012
TEST ACCURACY: 
0.2393

Epoch 6 Complete. Training Loss: 9.282049179077148
TEST ACCURACY: 
0.2416

Epoch 7 Complete. Training Loss: 9.302321434020996
TEST ACCURACY: 
0.2441

Epoch 8 Complete. Training Loss: 9.29590129852295
TEST ACCURACY: 
0.2479

Epoch 9 Complete. Training Loss: 9.013239860534668
TEST ACCURACY: 
0.2509

Epoch 10 Complete. Training Loss: 9.256378173828125
TEST ACCURACY: 
0.2542

Epoch 11 Complete. Training Loss: 9.025491714477539
TEST ACCURACY: 
0.2571

Epoch 12 Complete. Training Loss: 9.259920120239258
TEST ACCURACY: 
0.26

Epoch 13 Complete. Training Lo

In [202]:
num_test_labels = 10
start_point = 15
with tf.Session() as sess:
#     sess.run(init)
    saver.restore(sess,"../save/neuro_6-3.ckpt")
    
    weight= w63.eval()
    
    matches = tf.equal(tf.argmax(out_layer4, 1), tf.argmax(Y,1))
    acc = tf.reduce_mean(tf.cast(matches, tf.float32))
    
    print('TRAIN ACCURACY: ')
    print (sess.run(acc, feed_dict={X:mnist.train.images, Y:mnist.train.labels}))

    print('TEST ACCURACY: ')
    print (sess.run(acc, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))
    print('\n')

INFO:tensorflow:Restoring parameters from ../save/neuro_6-3.ckpt
TRAIN ACCURACY: 
0.2897818
TEST ACCURACY: 
0.3015




In [203]:
'''
training from previous

TRAIN ACCURACY: 
0.2897818
TEST ACCURACY: 
0.3015

training individually

TRAIN ACCURACY: 
0.28234544
TEST ACCURACY: 
0.2928


without training

TRAIN ACCURACY: 
0.1942909
TEST ACCURACY: 
0.201
'''

'\ntraining from previous\n\nTRAIN ACCURACY: \n0.2897818\nTEST ACCURACY: \n0.3015\n\ntraining individually\n\nTRAIN ACCURACY: \n0.28234544\nTEST ACCURACY: \n0.2928\n\n\nwithout training\n\nTRAIN ACCURACY: \n0.1942909\nTEST ACCURACY: \n0.201\n'

In [204]:
weight

array([[-0.49467096,  0.00711154, -0.05962576],
       [ 0.02446536, -0.09640595,  0.00109969],
       [ 0.06061416, -0.67002356,  0.14892067],
       [-0.80082935, -0.28167418, -0.6794569 ],
       [ 0.23381503, -0.595274  ,  0.17571352],
       [-0.18492433,  0.04384812, -0.24284588]], dtype=float32)

In [205]:
with tf.Session() as sess:
    sess.run(init)
    print(w63.eval())

[[-0.08728178  0.08770096 -0.06964457]
 [-0.14929977  0.1660394  -0.1005696 ]
 [ 0.34475893 -0.3366989   0.2863478 ]
 [-0.5960531   0.04331887 -1.1192211 ]
 [-0.02069826 -0.19160683 -0.26256895]
 [-0.2285434   0.01580529 -0.43007237]]


In [206]:
print(weight.shape)
np.save('../save/weights/ngw6-3.npy',weight)

(6, 3)
